In [3]:
import pandas as pd

# Importer le DataFrame propre depuis le fichier CSV
df = pd.read_csv('data.csv')


/tmp/ipykernel_8799/285193129.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


In [13]:
import pandas as pd

# Remplacer les valeurs non numériques par 0
df['NewExist'] = df['NewExist'].fillna(0)
df['UrbanRural'] = df['UrbanRural'].fillna(0)


# Convertir la colonne en type entier
df['NewExist'] = df['NewExist'].astype(int)
df['NewExist'].astype(int)

df['UrbanRural'] = df['UrbanRural'].astype(int)
df['UrbanRural'].astype(int)

df.dtypes

NAICS                 int64
ApprovalFY           object
Term                  int64
NewExist              int64
FranchiseCode         int64
UrbanRural            int64
RevLineCr            object
LowDoc               object
MIS_Status            int64
ApprovalDate_Year     int64
dtype: object

In [52]:
df.head(3)
df.drop(['ApprovalFY', 'ApprovalDate_Year'], axis=1)

#traiter la colonen NAICS pour qu'elle ne contienne que les 2 premiers chiffres des valeurs NAICS

df['NAICS_two_digit'] = (df['NAICS'] / 10000 ).astype(int)
df.drop(['NAICS'], axis=1)
df

,NAICS,ApprovalFY,Term,NewExist,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,ApprovalDate_Year,NAICS_two_digit
0,451120,1997,84,2,1,0,N,N,0,1997,45
1,722410,1997,60,2,1,0,N,N,0,1997,72
2,621210,1997,180,1,1,0,N,N,0,1997,62
3,0,1997,60,1,1,0,N,N,0,1997,0
4,0,1997,240,1,1,0,N,N,0,1997,0
...,...,...,...,...,...,...,...,...,...,...,...
897162,451120,1997,60,1,1,0,NaN,NaN,0,1997,45
897163,451130,1997,60,1,1,0,Y,Y,0,1997,45
897164,332321,1997,108,1,1,0,N,N,0,1997,33
897165,0,1997,60,1,1,0,N,N,1,1997,0


,NAICS,ApprovalFY,Term,NewExist,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,ApprovalDate_Year,NAICS_two_digit
0,451120,1997,84,2,1,0,N,N,0,1997,45
1,722410,1997,60,2,1,0,N,N,0,1997,72
2,621210,1997,180,1,1,0,N,N,0,1997,62
3,0,1997,60,1,1,0,N,N,0,1997,0
4,0,1997,240,1,1,0,N,N,0,1997,0
...,...,...,...,...,...,...,...,...,...,...,...
897162,451120,1997,60,1,1,0,NaN,NaN,0,1997,45
897163,451130,1997,60,1,1,0,Y,Y,0,1997,45
897164,332321,1997,108,1,1,0,N,N,0,1997,33
897165,0,1997,60,1,1,0,N,N,1,1997,0


In [43]:
# Modèle

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Visualisation

from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [44]:
#separer dataset en features et target
X = df.drop('MIS_Status', axis=1)
y = df['MIS_Status']

In [22]:
df.columns

Index(['NAICS', 'ApprovalFY', 'Term', 'NewExist', 'FranchiseCode',
       'UrbanRural', 'RevLineCr', 'LowDoc', 'MIS_Status', 'ApprovalDate_Year'],
      dtype='object')

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cat_col = ['RevLineCr', 'LowDoc', 'FranchiseCode' ]
num_col = ['NAICS', 'Term', 'NewExist', 'UrbanRural']

In [29]:
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline : prétraitement + modèle RandomForest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier())])

pipeline.fit(X_train, y_train)

# performance du modèle
score_tr = pipeline.score(X_train, y_train)
score_te = pipeline.score(X_test, y_test)

print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)


Score du modèle : 0.9202770935274251


### Hyperparamétres du modéle

* Random Forest
     - max_depth
     - min_sample_split
     - max_leaf_nodes
     - min_samples_leaf
     - n_estimators
     - max_sample (bootstrap sample)
     - max_features


In [ ]:
# Tests

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline : prétraitement + modèle RandomForest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier())])

pipeline.fit(X_train, y_train)

# performance du modèle
score_tr = pipeline.score(X_train, y_train)
score_te = pipeline.score(X_test, y_test)

print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from scipy.stats import randint
import numpy as np

# Assurez-vous que cat_col et num_col contiennent les noms des colonnes catégorielles et numériques

# random search cv
hyper_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],  # Ajoutez les stratégies d'imputation numérique ici si nécessaire
    'classifier_max_depth': list(np.arange(10, 100, step=10)) + [None],
    'classifier__n_estimators': np.arange(10, 500, step=50),
    'classifier__max_features': randint(1, 7),
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__min_samples_leaf': randint(1, 4),
    'classifier__min_samples_split': np.arange(2, 10, step=2)
}

# Création du preprocessor pour gérer les deux types de colonnes
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline incluant le prétraitement et le modèle RandomForest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier())])

random_cv = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=hyper_grid,
    cv=3,
    n_iter=5,
    scoring='accuracy',  # ou une autre métrique que vous souhaitez évaluer
    n_jobs=-1,
    return_train_score=True,
    random_state=42
)

# Fit du RandomizedSearchCV
random_cv.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print("Meilleurs paramètres:", random_cv.best_params_)

# Performance du meilleur modèle trouvé
score_tr = random_cv.best_estimator_.score(X_train, y_train)
score_te = random_cv.best_estimator_.score(X_test, y_test)

print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)
